In [1]:
!pip install bitsandbytes accelerate trl peft datasets unsloth -qqU

In [2]:
!sudo apt update
!sudo apt install -y build-essential

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 257 kB in 2s (137 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
52 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping ac

In [3]:
import pandas as pd
import numpy as np
import ast
import re

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import Dataset
from peft import get_peft_model, PeftModel, PeftConfig, PromptTuningConfig, TaskType, PromptTuningInit
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TextStreamer

from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, standardize_sharegpt, train_on_responses_only

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## NEXT

In [4]:
path = "/content/"

In [5]:
train_df = pd.read_csv(path + 'new_df_for_model.csv')
train_df = train_df.reset_index(drop=True)

In [6]:
train_df.sample(2)

,Unnamed: 0,id,task_id,description,input,output,student_solution,author_solution,author_comment,type,prompt
342,342,741,47,"Лолита изучает, какие студии дизайна чаще всег...","[nan, nan, nan]","['«Экспериментус», «СделаемЗавтра», «Айсбергус...","punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...","punctuation = '!#$%&()*+,-./:;<=>?@^_'\n\ndata...",Ошибка в открытых и скрытых тестах. \n\nВаш ко...,"['open', 'closed', 'closed']","<Task description>: Лолита изучает, какие студ..."
285,285,610,38,Реализуйте следующую программу:\n\nЗадан пусто...,['2 место – Nightmares (саунд)\n1 место – Анти...,"[""['1 место – Антиборщевик (айдентика)', '2 ме...",winners = []\n\n# ваш код\n\nwhile True:\n ...,winners = []\n\n# ваш код\n\nwhile True:\n ...,Ошибка в открытых и скрытых тестах. \n\nВаш ко...,"['open', 'open', 'closed', 'closed']",<Task description>: Реализуйте следующую прогр...


In [7]:
system_prompt = """Ты преподаватель Высшей школы экономики. Твоя задача, получив ЗАДАЧУ и РЕШЕНИЕ дать подсказку ученику, где находится ошибка в его РЕШЕНИИ. Прямо напиши где ошибка.

Не пиши большой ответ. Пиши только подсказку, которая поможет студенту двигаться в правильном направлении, но не видеть весь ответ сразу. Люди не должны догадаться, что ты робот.

Пример (Твоя задача отвечать так же лаконично и без конкретного решения, как и после <Комментарий эксперта>:):
<Task description>: Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:

Даны два списка logo_project и cite_project с кодами используемых цветов (строки).
В переменную color считывается код цвета (строка). Этот код уже написан.
Программа должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True.
В остальных случаях программа печатает False.
<Example 1>:
<Тип теста 1>: open
<Ввод>: #a7f0ca
<Вывод>: True
<Example 2>:
<Тип теста 2>: open
<Ввод>: #e4e3b3
<Вывод>: False
<Example 3>:
<Тип теста 3>: open
<Ввод>: #a7a8f0
<Вывод>: False
<Example 4>:
<Тип теста 4>: closed
<Ввод>: #c0ced7
<Вывод>: False
<Example 5>:
<Тип теста 5>: closed
<Ввод>: #a7f0ca
<Вывод>: True
<Решение студента>: logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and color in cite_project:
    print(True)
else:
    print(False)
<Идеальное решение>: logo_project = ['#a7a8f0', '#a7f0ca', '#b3b4e4', '#e4b3cd', '#e4e3b3', '#c0ced7']
cite_project = ['#e4e3b3', '#a7a8f0', '#ccb1e6', '#b4f99e', '#f9b59e', '#c0ced7']

color = input()

if color in logo_project and not(color in cite_project):
    print(True)
else:
    print(False)


<Комментарий эксперта>: Ошибка в открытых тестах.

Обратите внимание на неверный оператор сравнения — необходимо проверить, что цвет не находится в списке cite_project.
"""

conversations = []
for index, row in train_df.iterrows():
  description = row['description']
  rowie_input = row['input'].replace('nan', "'-'")
  rowie_output = row['output'].replace('nan', "'-'")
  row_input = ast.literal_eval(rowie_input)
  row_output = ast.literal_eval(rowie_output)
  row_type = ast.literal_eval(row['type'])

  res = []
  for i in range(len(row_input)):
    res.append(f"\n<Example {i+1}>: \n<Тип теста {i+1}>: {row_type[i]} \n<Ввод>: {row_input[i]} \n<Вывод>: {row_output[i]}")
  result = ' '.join(res)
  student_solution = row['student_solution']
  author_solution = row['author_solution']
  author_comment = row['author_comment']
  prompt = f"<Task description>: {description} {result} \n<Решение студента>: {student_solution} \n<Идеальное решение>: {author_solution} \n\n\n<Комментарий эксперта>: "
  messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": author_comment}
    ]

  conversations.append(messages)

In [8]:
train_df['conversations'] = conversations

In [9]:
train_df.shape

(347, 12)

In [10]:
train_df.iloc[0]['conversations']

[{'role': 'system',
  'content': "Ты преподаватель Высшей школы экономики. Твоя задача, получив ЗАДАЧУ и РЕШЕНИЕ дать подсказку ученику, где находится ошибка в его РЕШЕНИИ. Прямо напиши где ошибка.\n\nНе пиши большой ответ. Пиши только подсказку, которая поможет студенту двигаться в правильном направлении, но не видеть весь ответ сразу. Люди не должны догадаться, что ты робот.\n\nПример (Твоя задача отвечать так же лаконично и без конкретного решения, как и после <Комментарий эксперта>:):\n<Task description>: Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:\n\nДаны два списка logo_project и cite_project с кодами используемых цветов (строки).\nВ переменную color считывается код цвета (строка). Этот код уже написан.\nПрограмма должна проверять, что код цвета color есть только в списке logo_project, и если да, то печатать True.\nВ остальных случаях программа печатает False.\n<Example 1>:\n<Тип т

In [11]:
train_df.to_csv('new2_df_for_model.csv')

## Training args

In [12]:
max_seq_length = 2048
dtype = None
load_in_4bit = False

In [13]:
model_name = 'Vikhrmodels/Vikhr-Llama-3.2-1B-Instruct' # Vikhrmodels/Vikhr-Nemo-12B-Instruct-R-21-09-24

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [14]:
!nvidia-smi

Wed Oct 16 09:30:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0              28W /  70W |   1175MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
tuning_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.RANDOM,
    num_virtual_tokens=20, # количество добавленных обучаемых токенов
    tokenizer_name_or_path=model_name
)

model = get_peft_model(model, tuning_config)

In [16]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [17]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return {"text": texts}

In [18]:
dataset = Dataset.from_pandas(train_df)

In [19]:
dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'task_id', 'description', 'input', 'output', 'student_solution', 'author_solution', 'author_comment', 'type', 'prompt', 'conversations'],
    num_rows: 347
})

In [20]:
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/347 [00:00<?, ? examples/s]

Map:   0%|          | 0/347 [00:00<?, ? examples/s]

In [21]:
dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'task_id', 'description', 'input', 'output', 'student_solution', 'author_solution', 'author_comment', 'type', 'prompt', 'conversations', 'text'],
    num_rows: 347
})

In [22]:
dataset['text'][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы преподаватель Высшей школы экономики. Твоя задача, получив ЗАДАЧУ и РЕШЕНИЕ дать подсказку ученику, где находится ошибка в его РЕШЕНИИ. Прямо напиши где ошибка.\n\nНе пиши большой ответ. Пиши только подсказку, которая поможет студенту двигаться в правильном направлении, но не видеть весь ответ сразу. Люди не должны догадаться, что ты робот.\n\nПример (Твоя задача отвечать так же лаконично и без конкретного решения, как и после <Комментарий эксперта>:):\n<Task description>: Реализуйте программу, которая проверит, что цвет используется только в проекте по созданию логотипа, но не в проекте по созданию дизайна сайта:\n\nДаны два списка logo_project и cite_project с кодами используемых цветов (строки).\nВ переменную color считывается код цвета (строка). Этот код уже написан.\nПрограмма должна проверять, что код цвета color есть только в списке logo_project, 

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #num_train_epochs = 2,
        max_steps = 120, ################## НАДО ЭТО ПОКРУТИТЬ ИЛИ РАСКОММЕНТИТЬ num_train_epochs
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/347 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [24]:
# сделаем так, чтобы лоссы считались только на ответах модели
# и не считались на остальных входных данных

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/347 [00:00<?, ? examples/s]

In [34]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Trainable parameter: {name}")

Trainable parameter: prompt_encoder.default.embedding.weight


In [25]:
trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:452: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss
1,8.039000
2,8.220800
3,7.486500
4,7.617000
5,7.405000
6,7.900500
7,7.793400
8,7.801600
9,7.522400
10,7.353200


## Preparing test dataset

In [26]:
test_df = pd.read_csv(path + 'new_test_df_for_inference.csv')

In [27]:
test_df.sample(2)

,Unnamed: 0,id,task_id,description,input,output,student_solution,author_solution,author_comment,type,prompt
105,105,274,17,"Реализуйте программу, которая определит поселк...","['кабинет', 'гараж', 'гостинная', 'гостевой са...","['Гринфилд/Гринфилд', nan, 'Театральный кварта...","room = input()\n\nresidences = []\nfor key, va...",studios = {\n \n 'Rutemple Studio': {'Ск...,NaN,"['open', 'open', 'open', 'closed', 'closed', '...","<Task description>: Реализуйте программу, кото..."
192,192,484,31,Роман анализирует информаицю о переведенных те...,"['75000', '82100', '30000', '1000000']","['2004,французский,иврит,78918 слов\n1998,фран...","words = int(input())\n\nopen('info_texts.csv',...",words = int(input())\n\nwith open('info_texts....,NaN,"['open', 'closed', 'closed', 'closed']",<Task description>: Роман анализирует информаи...


## Inference

In [28]:
from tqdm import tqdm

In [29]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(128256, 2048)
      (layers): ModuleList(
        (0-15): 16 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
            (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
            (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
            (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
            (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm((2048,), eps

In [35]:
#num_virtual_tokens = 20
#def add_virtual_tokens(input_ids, num_virtual_tokens):
#    virtual_tokens = torch.zeros((input_ids.size(0), num_virtual_tokens), dtype=torch.long).to(input_ids.device)
#    return torch.cat([virtual_tokens, input_ids], dim=1)

def preprocess_answer(text):
  cleaned_text = re.sub(r"<\|.*?\|>|assistant", "", text)
  cleaned_text = cleaned_text.strip()
  return cleaned_text

answers_test = []
for index, row in tqdm(test_df[:10].iterrows()):
  prompt = row['prompt']
  messages = [
    {"role": "user", "content": prompt},
  ]

  inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
  ).to("cuda")
  #input_ids_with_virtual_tokens = add_virtual_tokens(inputs, num_virtual_tokens)

  outputs = model.generate(
      input_ids=inputs, #input_ids_with_virtual_tokens,
      max_new_tokens=200,
      use_cache=True
  )
  answer = tokenizer.batch_decode(outputs)
  id_header = answer[0].find('<|start_header_id|>assistant<|end_header_id|>')
  text = answer[0][id_header:]
  answer = preprocess_answer(text)
  answers_test.append(answer)

0it [00:00, ?it/s]


ValueError: The following `model_kwargs` are not used by the model: ['num_logits_to_keep'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
len(answers_test)

10

In [ ]:
answer_test = pd.DataFrame(answers_test)

In [ ]:
for i in range(2):
    print(answer_test.iloc[i+2][0])

Ваш код имеет неверный синтаксис f-строки. Так, вы не можете использовать знак минус непосредственно в f-строке. Попробуйте заключить выражение для вычисления скидки в кавычки.Вычисление скидки следует выполнить заранее.Ваш код должен вычислять скидку, а затем использовать переменную, содержащую значение скидки.Вот исправленный код.

```python
# Ваш код выше

# Вычисление стоимости проекта со скидкой. Сначала вычисляем значение скидки.Обратите внимание, что мы не вычитаем скидку.Скорректируем условие.<
Ваш код некорректно выполняет условие задания. Так, в данном коде, в f-строке, которая выводит данные, используется некорректная переменная. Так, в f-строке, чтобы вывести некорректированную стоимость проекта, необходимо использовать не значение переменной discount, а значение money. Так же, в вычислении скиднутой стоимости проекта, некорректно использован знак оператора. Так, чтобы скорректировать значение money на величину скидки, необходимо использовать не вычитание, а умножение money

In [ ]:
answer_test.to_csv('inference2time_with_training.csv')

In [ ]:
from typing import Callable

import pandas as pd
import torch
from transformers import BertModel, BertTokenizer

print("Loading models...", end="")
model_name = "DeepPavlov/rubert-base-cased-sentence"
emb_tokenizer = BertTokenizer.from_pretrained(model_name)
emb_model = BertModel.from_pretrained(model_name)
print("OK")


def get_sentence_embedding(sentence: str) -> torch.Tensor:
    inputs = emb_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = emb_model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    return embedding


def string2embedding(string: str) -> torch.Tensor:
    return torch.Tensor([float(i) for i in string.split()])


def embedding2string(embedding: torch.Tensor) -> str:
    return " ".join([str(i) for i in embedding.tolist()])


def generate_submit(test_solutions: str, predict_func: Callable, save_path: str, use_tqdm: bool = True) -> None:
    bar = range(len(test_solutions))
    if use_tqdm:
        import tqdm

        bar = tqdm.tqdm(bar, desc="Predicting")

    submit_df = pd.DataFrame(columns=["solution_id", "author_comment", "author_comment_embedding"])
    for i in bar:
        idx = test_solutions.id.iloc[i]
        solution_row = test_solutions.prompt.iloc[i]
        text = predict_func(solution_row)
        embedding = embedding2string(get_sentence_embedding(text))
        submit_df.loc[i] = [idx, text, embedding]
    submit_df.to_csv(save_path, index=False)

Loading models...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


OK


In [ ]:
def preprocess_answer(text):
  cleaned_text = re.sub(r"<\|.*?\|>|assistant", "", text)
  cleaned_text = cleaned_text.strip()
  return cleaned_text

def predict(prompt):
    messages = [
      {"role": "user", "content": prompt},
    ]
    inputs = tokenizer.apply_chat_template(
      messages,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt",
    ).to("cuda")
    outputs = model.generate(input_ids=inputs, use_cache=True)
    answer = tokenizer.batch_decode(outputs)
    id_header = answer[0].find('<|start_header_id|>assistant<|end_header_id|>')
    text = answer[0][id_header:]
    return preprocess_answer(text)

In [ ]:
generate_submit(
        test_solutions=test_df,
        predict_func=predict,
        save_path="submit_2_pobedniy.csv",
        use_tqdm=True,
)

Predicting:   0%|          | 0/325 [00:00<?, ?it/s]


ValueError: Cannot use apply_chat_template() because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
df = pd.read_csv(path + "submit_2_pobedniy.csv")
df.head()